# Кейс ЦБ РФ

## Подключение нужных библиотек

In [2]:
import numpy as np
import torch

## Проверяем наличие GPU

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используем устройство:', device)

Используем устройство: cuda


## Создаем Dataset для дальнейшей работы

In [5]:
class CbrDataset(torch.utils.data.Dataset):
    def __init__(self, inp_file, out_file):
        self.x_data = np.load(inp_file)
        self.y_data = np.load(out_file)
        
    def __len__(self):
        return len(self.x_data)
        
    def __getitem__(self, idx):
        inputs = self.x_data[idx]
        outputs = self.y_data[idx] 
        return (inputs, outputs)  

## Создаем нужную нейросеть

In [7]:
class CbrNet(torch.nn.Module):
  def __init__(self):
    super(CbrNet, self).__init__()
    self.hid1 = torch.nn.Linear(600, 300)  
    self.hid2 = torch.nn.Linear(300, 100)
    self.oupt = torch.nn.Linear(100, 15)

    torch.nn.init.xavier_uniform_(self.hid1.weight) 
    torch.nn.init.zeros_(self.hid1.bias)
    torch.nn.init.xavier_uniform_(self.hid2.weight)
    torch.nn.init.zeros_(self.hid2.bias)
    torch.nn.init.xavier_uniform_(self.oupt.weight)
    torch.nn.init.zeros_(self.oupt.bias)

  def forward(self, x):
    z = torch.tanh(self.hid1(x)) 
    z = torch.tanh(self.hid2(z))
    z = self.oupt(z)  
    return z


## Обучение модели

In [9]:
def train(model, ds, bs, lr, me, le):
    train_ldr = torch.utils.data.DataLoader(ds, batch_size=bs, shuffle=True)
    loss_func = torch.nn.L1Loss()  
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(0, me):
        epoch_loss = 0.0  # for one full epoch
        for (b_idx, batch) in enumerate(train_ldr):
            X = batch[0]
            y = batch[1]
            optimizer.zero_grad()
            oupt = model(X)
            loss_val = loss_func(oupt, y)  
            epoch_loss += loss_val.item()  
            loss_val.backward()  
            optimizer.step()     

        if epoch % le == 0:
            print(f"epoch = {epoch: 4d}  |  loss = {epoch_loss:0.4f}")
            torch.save(model, f".\results\model_{epoch:4d}")
    
    #torch.save(model, f".\results\model_final.pt")
        

## Главная программа

In [ ]:
# 0. get started
print("\nРешение кейса ЦБ России с использованием PyTorch")
np.random.seed(42) 
torch.manual_seed(42) 

# 1. Создание объектов Dataset и DataLoader
print("\nЗагрузка обучающего Dataset")
train_input_file = f".data\y_smp_train.npy"
train_output_file = f".data\pars_smp_train.npy"
train_dataset = CbrDataset(train_input_file, train_output_file)
trainlloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True) # RSF

data_batch, labels_batch = next(iter(trainloader))
print(data_batch.size())
# out: torch.Size([16, 3, 32, 32])
print(labels_batch.size())
# out: torch.Size([16])

#  test_file = ".\\Data\\boston_test.txt"
#  test_ds = BostonDataset(test_file)

# 2. Создание модели
print("\nСоздание 600-(300-100)-15 регрессионной сети")
net = CbrNet().to(device)
net.train()

# 3. Обучаем модель
bs=10 # размер батча
lr=0.005 
me=5000 # Максимальное количество эпох
le=1000 # Точки выдачи отчета

print(f"\nbatch size = {bs}", end = " ")
print("функция потерь = L1Loss()", end = " ")
print("оптимизатор = Adam", end = " ")
print(f"learn rate = {lf}", end = " ")
print(f"количество эпох = {me}")

print("\nНачало обучения ")
train(net, train_ds, bs, lr, me=500, le=100)
print("Завершение")

# 4. Точность модели
net.eval()
acc_train = accuracy(net, train_ds, 0.20)
print(f"\nAccuracy on train (within 0.20) = {acc_train:%0.4f}")
acc_test = accuracy(net, test_ds, 0.20)
print("Accuracy on test (within 0.20) = %0.4f " % acc_test)

# 5. Сохранение модели
torch.save(model, f".\results\model_final.pt")

# 6. Использование модели
  print("\nPredicting normalized (poverty, price) first train")
  print("Actual (poverty, price) = (0.0914, 0.2160) ")

  x = np.array([0.000273, 0.000, 0.0707, -1, 0.469,
    0.6421, 0.789, 0.049671, 0.02, 0.242, 0.178,
    0.39690], dtype=np.float32)
  x = T.tensor(x, dtype=T.float32)

  with T.no_grad():
    oupt = net(x)
  print("Predicted poverty price = %0.4f %0.4f " % \
    (oupt[0], oupt[1]))

  print("\nEnd demo ")


In [ ]:
# 1. Создание объектов Dataset и DataLoader
print("Загрузка обучающего Dataset")
train_input_file = f"./data/y_smp_train.npy"
train_output_file = f"./data/pars_smp_train.npy"
train_dataset = CbrDataset(train_input_file, train_output_file)
trainlloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True) # RSF

data_batch, labels_batch = next(iter(trainloader))
print(data_batch.size())
# out: torch.Size([16, 3, 32, 32])
print(labels_batch.size())
# out: torch.Size([16])


Загрузка обучающего Dataset
